In [1]:
import tensorflow as tf

# MNIST手写数字

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("/path/to/MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /path/to/MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /path/to/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /path/to/MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /path/to/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives 

In [7]:
print(mnist.train.num_examples)
print(mnist.validation.num_examples)
print(mnist.test.num_examples)

55000
5000
10000


In [9]:
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99 #学习率衰减率
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99 #滑动平均衰减率

In [25]:
def inference(input_tensor, avg_class, weights1, biases1,
             weights2, biases2):
    if avg_class == None: #无滑动平均类
        layers1 = tf.nn.relu(tf.matmul(input_tensor, weights1)
                            + biases1)
        return tf.matmul(layers1, weights2) + biases2
    else:
        layers1 = tf.nn.relu(tf.matmul(input_tensor, 
                                       avg_class.average(weights1))
                            + avg_class.average(biases1))
        return (tf.matmul(layers1, avg_class.average(weights2))
                + avg_class.average(biases2))

In [40]:
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name="x-input")
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name="y-input")
    weights1 = tf.Variable(tf.truncated_normal(
    [INPUT_NODE, LAYER1_NODE], stddev = 0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    weights2 = tf.Variable(tf.truncated_normal(
    [LAYER1_NODE, OUTPUT_NODE], stddev = 0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    #不使用滑动平均
    y = inference(x, None, weights1, biases1, weights2, biases2)
    #滑动平均
    global_step = tf.Variable(0, trainable=False)
    variable_averages = tf.train.ExponentialMovingAverage(
    MOVING_AVERAGE_DECAY, global_step)
    variable_averages_op = variable_averages.apply(
    tf.trainable_variables())
    average_y = inference(
        x, variable_averages, weights1, biases1, weights2, biases2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=y, labels=tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weights1)+regularizer(weights2)
    loss = cross_entropy_mean + regularization
    learning_rate = tf.train.exponential_decay(
    LEARNING_RATE_BASE, global_step, mnist.train.num_examples/BATCH_SIZE,
    LEARNING_RATE_DECAY)
    train_step=tf.train.GradientDescentOptimizer(learning_rate)\
    .minimize(loss, global_step=global_step)
    #更新滑动平均值
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name="train")
    
    correct_prediction = tf.equal(tf.argmax(average_y, 1),
                                  tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x:mnist.validation.images,
                        y_:mnist.validation.labels}
        test_feed = {x:mnist.test.images,
                     y_:mnist.test.labels}
        for i in range(TRAINING_STEPS):
            if i%1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print(i,validate_acc)
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict = {x:xs, y_:ys})
        test_acc = sess.run(accuracy, feed_dict = test_feed)
        print("OK",TRAINING_STEPS, test_acc)

In [41]:
def main(argv=None):
    train(mnist)

In [42]:
if __name__ == "__main__":
    tf.app.run()

0 0.0736
1000 0.9774
2000 0.9818
3000 0.9828
4000 0.9836
5000 0.9844
6000 0.9842
7000 0.9848
8000 0.9842
9000 0.9838
10000 0.986
11000 0.9848
12000 0.9854
13000 0.9846
14000 0.9858
15000 0.986
16000 0.9856
17000 0.9854
18000 0.9858
19000 0.9856
20000 0.9864
21000 0.9862
22000 0.9854
23000 0.986
24000 0.9864
25000 0.986
26000 0.986
27000 0.986
28000 0.986
29000 0.9862
OK 30000 0.9848


SystemExit: 

c:\users\tulon\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
119/297